## Import Modules

In [1]:
import pandas as pd
import re
import os

## Adding Directories

``` 
We have 4 paths here,

    1. path_dir is the source where all the sub directories are located
    2. src_dir is the source for text input files
    3. dst_tmp_ir is destination path for temporary csv files
    4. dst_dir is the destination path for final csv files

```

In [4]:
path_dir = r'C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small'
src_dir = os.path.join(path_dir,'txt files')
dst_tmp_dir = os.path.join(path_dir,'csv files temp')
dst_dir = os.path.join(path_dir,'csv files')

lst_files = os.listdir(src_dir)

## Checking Files in the Source file

In [5]:
iter_tp = 0
for iter_fl in lst_files:
    print(iter_tp,iter_fl)
    iter_tp+=1

0 personalized-dialog-task1-API-calls-dev.txt
1 personalized-dialog-task1-API-calls-trn.txt
2 personalized-dialog-task1-API-calls-tst-OOV.txt
3 personalized-dialog-task1-API-calls-tst.txt
4 personalized-dialog-task2-API-refine-dev.txt
5 personalized-dialog-task2-API-refine-trn.txt
6 personalized-dialog-task2-API-refine-tst-OOV.txt
7 personalized-dialog-task2-API-refine-tst.txt
8 personalized-dialog-task3-options-dev.txt
9 personalized-dialog-task3-options-trn.txt
10 personalized-dialog-task3-options-tst-OOV.txt
11 personalized-dialog-task3-options-tst.txt
12 personalized-dialog-task4-info-dev.txt
13 personalized-dialog-task4-info-trn.txt
14 personalized-dialog-task4-info-tst-OOV.txt
15 personalized-dialog-task4-info-tst.txt
16 personalized-dialog-task5-full-dialogs-dev.txt
17 personalized-dialog-task5-full-dialogs-trn.txt
18 personalized-dialog-task5-full-dialogs-tst-OOV.txt
19 personalized-dialog-task5-full-dialogs-tst.txt


## Run Single File

In [58]:
fl_no = 0
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\full\txt files\personalized-dialog-task1-API-calls-dev.txt


## Creating temp CSV file

``` temp csv file contains profile,knowldgebase as seperate columns i.e we are yet to add tags such as <user> <profile> <query> <system> etc.. ```

In [59]:
df = pd.DataFrame([])  

## Initializing df to add data to

path_to_use = path_final
ct = 0

with open(path_to_use) as topo_file:

    ## Open the text file from given path

    cnt = 0
    cnt_sub = 0

    ## cnt_sub is a Counter 

    ar_3,ar_4 = [],[]
    ar_1,ar_2 = [],[]

    ## we are initializing 4 arrays, each array stores the following
    ## Gender, age-group , Call , Response , Counter
    ## Gender (ar_1) and age-group (ar_2) are self explainable
    ## Call has the question asked (ar_3)
    ## Response is the answer given (ar_4)
    ## Counter is to understand where a new conversation starts

    for line in topo_file:

        ## Go through each line of the text file

        line = line.strip()    
        if not line.strip(): continue

        ## If the line is empty continue to next line

        init_num = re.findall(r'\d+',line)        

        ## identifying the number before each line

        if init_num[0]=='1':

            ## The first line has the profile, So Identifying 1
            
            line_splits = line[1:].split()

            ## We split the line 1 to get Gender and age

            ar_1.append(line_splits[0])

            ## Appending Gender to array 1

            ar_2.append(line_splits[1])

            ## Appending Age to array 2

            ar_5 = cnt

            ## Since Profile is to be appended to all examples, 
            ## we have to identify number of Call and Responses in a SIngle Conversation

            cnt+=1

            if len(ar_3)==0: continue

            ## Check if we are on line 1 of 1st conversation.
            ## if we are skip for now
            
            data = list(
                zip(
                    [ar_1[0]]*len(ar_3),
                    [ar_2[0]]*len(ar_3),                    
                    ar_3,
                    ar_4,
                    [ar_5]*len(ar_3)       
                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_3,ar_4 = [],[]
            ar_1.pop(0)
            ar_2.pop(0)

            ## Add the data of first Conversation to the main dataframe defined
            ## ReSet the counter, Call and Response arrays
            ## Since Gender and age-group are single values pop them from begining , essentially resetting them  
            continue


        line_splits = line[2:].split('\t')
        ar_3.append(line_splits[0])
        ar_4.append(line_splits[1])

        ## Splitting Call and Response
        ## Addinig Call and Response to arrays 3 & 4

        cnt_sub+=1

        ## Incrementing Counter 

df = df.rename(columns={
     0: 'Gender',
     1: 'Age',
     2: 'Call',
     3: 'Response',
     4: 'Count'
     })

## Adding Column Names to our dataframe 

print(df.shape)
fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)

## getting file name and appending it to the destination path

df.to_csv(path_dst_tmp,index=False)

## Writing File to temp csv file

(36086, 5)


## Final CSV File

In [60]:
TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    ' <query> ']


## Defining tags we have to use in our final CSV file


mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''

for i in range(len(df)):    
    
    ## Go through each line of the temp CSV file
    ## From the Counter Column Identify beginning of each Conversation 

    if df.iloc[i][4]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][4]
    
    a2 = df.iloc[i][2]
    a3 = df.iloc[i][3]    

    ## get call and Response Column


    call_string = prev_string + TAGS[4]+ a2    
    prev_string = prev_string + TAGS[2]+  a2 + TAGS[3] + a3
    to_append = TAGS[0]+ line

    ## adding the tags wherever they are appropriate
    ## for the present line add only <Query> tag ( which is tag[4])
    ## for next set of lines in the conversation add <User> and <Sytem> tags ( tag[2],tag[3])
    ## we do this until the counter identifies a new conversation starting, at the beginning of the loop    


    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

    ## appending new Call and Responses to arrays

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 

## Creating new dataframe with new set of Call and Response columns

df['Profile'] = df['Gender'] + ' ' +df['Age']
ndf['Call'] = TAGS[0]+ df['Profile'] + TAGS[1] + ndf['Call']

## Since all Lines would have the Profile, adding the profile tag at the begining 
## of each line and appending it to the new df


print(ndf.shape)
fl_dst = lst_files[fl_no][:-3] + '.csv'
path_dst = os.path.join(dst_dir,fl_dst)

## getting file name and appending it to the destination path

ndf.to_csv(path_dst,index=False)
    
## Writing File to Final csv file

(36086, 2)


## Defining the above steps as a Functions

In [6]:
def convert_to_df(path_final):
    df = pd.DataFrame([])
    path_to_use = path_final
    ct = 0
    with open(path_to_use) as topo_file:
        cnt = 0
        cnt_sub = 0
        ar_3,ar_4 = [],[]
        ar_1,ar_2 = [],[]

        for line in topo_file:
            # if ct>120: break
            # ct+=1

            line = line.strip()    
            if not line.strip(): continue

            init_num = re.findall(r'\d+',line)        

            if init_num[0]=='1':

                #print(line)
                line_splits = line[1:].split()
                ar_1.append(line_splits[0])
                ar_2.append(line_splits[1])
                ar_5 = cnt
                cnt+=1

                if len(ar_3)==0: continue
                
                data = list(
                    zip(
                        [ar_1[0]]*len(ar_3),
                        [ar_2[0]]*len(ar_3),                    
                        ar_3,
                        ar_4,
                        [ar_5]*len(ar_3)       
                        ))

                df = df.append(data,ignore_index=True)
                cnt_sub = 0
                ar_3,ar_4 = [],[]
                ar_1.pop(0)
                ar_2.pop(0)
                continue

            line_splits = line[2:].split('\t')
            ar_3.append(line_splits[0])
            ar_4.append(line_splits[1])
            cnt_sub+=1

    df = df.rename(columns={
        0: 'Gender',
        1: 'Age',
        2: 'Call',
        3: 'Response',
        4: 'Count'
        })
    print(df.shape)
    fl_dst = lst_files[fl_no][:-3] + 'csv'
    path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
    df.to_csv(path_dst_tmp,index=False)
    return df

TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    ' <query> ']
    
def add_context(df):
    prev_count = 0
    txt = []
    for i in range(len(df)):    
        if df.iloc[i][4]!=prev_count:
            prev_count = df.iloc[i][4]
            txt.append('')
            continue
        txt.append(' <context>')  
    df['TAG'] = txt
    return df


def parse_from_df(df):
    c = 0
    mn_ar_1 = []
    mn_ar_2 = []
    mn_ar_3 = []
    prev_count = 0
    prev_string = ''
    for i in range(len(df)):    
        #if c > 100: break
        c+=1    

        if df.iloc[i][4]!=prev_count:
            prev_string = ''
            prev_count = df.iloc[i][4]
        
        a2 = df.iloc[i][2]
        a3 = df.iloc[i][3]    
        call_string = prev_string + TAGS[4]+ a2    
        prev_string = prev_string + TAGS[2]+ a2 + TAGS[3] + a3
        # print(a2)
        # print('call: ',call_string)
        # print('prev: ',call_string)
        # print(a3)
        mn_ar_1.append(call_string)
        mn_ar_2.append(a3)

    ndf = pd.DataFrame(            
                list(zip(
                    mn_ar_1,mn_ar_2                  
                    )),

                columns =[
                    'Call',
                    'Response'
                    ]
                    ) 

    df['Profile'] = df['Gender'] + ' ' +df['Age']
    ndf['Call'] = TAGS[0]+ df['Profile'] + df['TAG'] + ndf['Call']

    print(ndf.shape)
    fl_dst = lst_files[fl_no][:-3] + '.csv'
    path_dst = os.path.join(dst_dir,fl_dst)
    ndf.to_csv(path_dst,index=False)       

## A loop to get the index of Files to run 

In [8]:
for i in range(0,9):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    print(i,path_final)

0 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task1-API-calls-dev.txt
1 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task1-API-calls-trn.txt
2 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task1-API-calls-tst-OOV.txt
3 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task1-API-calls-tst.txt
4 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task2-API-refine-dev.txt
5 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task2-API-refine-trn.txt
6 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task2-API-refine-tst-OOV.txt
7 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task2-API-refine-tst.txt
8 C:\Users\ydjoe\Documents\nlp\perso

## Running the loop to preprocess all the similar files at once

In [9]:
for i in range(0,8):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    df = convert_to_df(path_final)
    df = add_context(df)
    parse_from_df(df)
    

(6008, 5)
(6008, 2)
(6016, 5)
(6016, 2)
(6014, 5)
(6014, 2)
(5931, 5)
(5931, 2)
(9502, 5)
(9502, 2)
(9488, 5)
(9488, 2)
(9450, 5)
(9450, 2)
(9453, 5)
(9453, 2)
